In [1]:

import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df = pd.read_csv('tracking_week_1.csv')

In [3]:
lineset = df[df['event']=='line_set']

In [5]:
motionpositions = players[['nflId','position']][(players['position'] == "WR") | (players['position'] == "RB") | (players['position'] == "TE") | (players['position'] == "CB") | (players['position'] == "OLB") | (players['position'] == "DE") | (players['position'] == "ILB") | (players['position'] == "FS") | (players['position'] == "SS")]

In [6]:
motionplayers = pd.merge(
    lineset,
    motionpositions,
    how = "left",
    left_on = ['nflId'],
    right_on = ['nflId']
)

In [7]:
motionplayers = motionplayers[motionplayers['position'].notna()]
motionplayers = motionplayers[['gameId','playId','nflId','x','y','position']]

,gameId,playId,nflId,x,y,position
0,2022091200,64,35459.0,51.88,29.07,SS
3,2022091200,64,42393.0,45.84,14.00,CB
4,2022091200,64,42403.0,40.59,18.37,OLB
5,2022091200,64,42412.0,39.06,14.49,WR
6,2022091200,64,42826.0,40.69,22.06,DE
...,...,...,...,...,...,...
44245,2022090800,3696,53459.0,25.73,35.49,DE
44246,2022090800,3696,53641.0,14.15,21.12,SS
44247,2022090800,3696,53678.0,28.46,43.94,WR
44248,2022090800,3696,54488.0,25.68,44.13,CB


In [8]:
offensive_players = motionplayers[motionplayers['position'].isin(["WR", "TE", "RB"])].copy()
defensive_players = motionplayers[~motionplayers['position'].isin(["WR", "TE", "RB"])].copy()


In [9]:
offensive_players['defender'] = None

In [10]:
for i in range(len(offensive_players)):
    game = offensive_players.iloc[i]['gameId']
    play = offensive_players.iloc[i]['playId']
    potentialDefenders = defensive_players[(defensive_players['gameId'] == game) & (defensive_players['playId'] == play)]
    closest = 99999999
    closestid = 0
    for j in range(len(potentialDefenders)):
        if abs(offensive_players.iloc[i]['y'] - potentialDefenders.iloc[j]['y']) < closest:
            closest = abs(offensive_players.iloc[i]['y'] - potentialDefenders.iloc[j]['y'])
            closestid = potentialDefenders.iloc[j]['nflId']
    offensive_players.loc[offensive_players.index[i], 'defender'] = closestid

In [13]:
offensive_players = pd.merge(
    offensive_players,
    plays[['gameId','playId','pff_manZone']],
    how="left",
    left_on = ['gameId','playId'],
    right_on = ['gameId','playId']
)

In [14]:
offensive_players = offensive_players[offensive_players['pff_manZone']=="Man"]

In [15]:
offensive_players

,gameId,playId,nflId,x,y,position,defender,pff_manZone
25,2022091200,201,40026.0,70.11,18.71,WR,42030.0,Man
26,2022091200,201,42412.0,70.07,39.44,WR,42393.0,Man
27,2022091200,201,46189.0,70.71,34.92,TE,35459.0,Man
28,2022091200,201,47847.0,70.76,10.03,WR,53438.0,Man
29,2022091200,201,47987.0,66.42,27.74,RB,47854.0,Man
...,...,...,...,...,...,...,...,...
9281,2022090800,3190,41290.0,71.53,7.35,WR,52647.0,Man
9282,2022090800,3190,43399.0,72.77,30.59,TE,40166.0,Man
9283,2022090800,3190,44881.0,72.39,16.98,WR,46190.0,Man
9284,2022090800,3190,47853.0,75.90,21.79,RB,46085.0,Man


In [17]:
linesetDefender = pd.merge(
    offensive_players,
    players[['nflId','position']],
    how="left",
    left_on = ['defender'],
    right_on = ['nflId']
)

In [20]:
linesetDefender=linesetDefender[['gameId','playId','nflId_x','nflId_y','position_y']]

In [21]:
linesetDefender.rename(columns={
    'nflId_x': 'routeRunnerId',
    'nflId_y': 'presnapDefenderId',
    'position_y': 'presnapDefenderPosition'
}, inplace=True)

C:\Users\bhaga\AppData\Local\Temp\ipykernel_11756\3035609452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  linesetDefender.rename(columns={


In [22]:
linesetDefender

,gameId,playId,routeRunnerId,presnapDefenderId,presnapDefenderPosition
0,2022091200,201,40026.0,42030,CB
1,2022091200,201,42412.0,42393,CB
2,2022091200,201,46189.0,35459,SS
3,2022091200,201,47847.0,53438,CB
4,2022091200,201,47987.0,47854,DE
...,...,...,...,...,...
2057,2022090800,3190,41290.0,52647,CB
2058,2022090800,3190,43399.0,40166,SS
2059,2022090800,3190,44881.0,46190,CB
2060,2022090800,3190,47853.0,46085,ILB


In [23]:
linesetDefender.to_csv('linesetDefender.csv',index=False)